# Assignment 1: NYC Taxi Data
## Machine Learning Model
Predict the **total fare amount** of a trip based for the last _3_ months of data (train data on the remaining dataset). The field _fare_amount_ can not be used as a feature in the model.

The final model will be assessed using the **RMSE** score.

In [27]:
import pandas as pd
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor, GeneralizedLinearRegression
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StringType

In [2]:
# Create a local spark session
spark = SparkSession.builder \
  .appName('nyc-taxi-model') \
  .getOrCreate()

In [3]:
# Set parameters 
target_field = "total_amount"
file_loc = "./output"
stages = []

In [56]:
def create_model_dataframe(file_loc, cat_cols, num_cols, tgt_col):
    # Read data from parquet
    df = spark.read.parquet(file_loc)
    
    # Select only required fields from source and rename target column
    if isinstance(tgt_col, list):
        select_cols = cat_cols + num_cols + tgt_col
    else:
        select_cols = cat_cols + num_cols + [tgt_col]   
    df = df.select(select_cols).withColumnRenamed(target_field, "target")
        
    return df

In [29]:
def create_model_dataframe_glm(file_loc, cat_cols, num_cols, tgt_col):
    # Read data from parquet
    df = spark.read.parquet(file_loc)
    
    # Select only required fields from source and rename target column
    if isinstance(tgt_col, list):
        select_cols = cat_cols + num_cols + tgt_col
    else:
        select_cols = cat_cols + num_cols + [tgt_col]   
    df = df.select(select_cols).withColumnRenamed(target_field, "label")
    
    df= df.withColumn("RatecodeID",df["RatecodeID"].cast(StringType()))
    
    return df

In [5]:
# https://www.timlrx.com/blog/feature-selection-using-feature-importance-score-creating-a-pyspark-estimator
def get_feature_importance(importance, dataset, features):
    list_extract = []
    
    for i in dataset.schema[features].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[features].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: importance[x])
    return(varlist.sort_values('score', ascending = False))

In [6]:
def get_train_dataset(df):
    df = df.filter((F.col("year") == 2015) | 
                   (F.col("year") == 2016) & (F.col("month").isin([1,2,3,4,5,6,7,8,9])))
    return df

In [7]:
def get_test_dataset(df):
    df = df.filter((F.col("year") == 2016) & (F.col("month").isin([10,11,12])))
    return df

In [54]:
# If model is passed, skip model fit portion and show test results

def run_random_forest_model(df, cat_cols, num_cols, trees=20, sub_sample=0.7, feature_imp=True, rf_model= False):
    stages = []
    
    # For category columns implement One Hot Encoding for each field
    for col in cat_cols:
        column_indexer = StringIndexer(inputCol=col, outputCol=f"{col}_ind")
        column_encoder = OneHotEncoderEstimator(inputCols=[f"{col}_ind"], outputCols=[f"{col}_ohe"])
        stages += [column_indexer, column_encoder]
    
    # Create a list of category fields that have been OHE
    cat_cols_ohe = [f"{col}_ohe" for col in cat_cols]
        
    # Instantiate a VectorAssembler of all categorical and number columns
    assembler = VectorAssembler(inputCols=cat_cols_ohe + num_cols, outputCol='features')
    
    # Add to stages list
    stages += [assembler]
    
    # Instantiate a pipeline with stages
    pipeline = Pipeline(stages=stages)

    # Fit the pipeline with model data frame
    pipeline_model = pipeline.fit(df)
        
    # Get test/train data
    train_data = get_train_dataset(df)
    test_data = get_test_dataset(df)
    
    #test_data.show(20)
    #train_data.show(20)

    
    # Apply the pipeline to the dataframe
    test_data = pipeline_model.transform(test_data)
    train_data = pipeline_model.transform(train_data)

    # Train a Random Forest model to predict the target field
    rf = RandomForestRegressor(featuresCol='features', labelCol='target', seed=77,
                               numTrees=trees, subsamplingRate=sub_sample)

    # if there is no model passed
    if not rf_model:
        # Use the training data to create a model
        rf_model = rf.fit(train_data)
        
    # Test the model using 3 months of trip data
    rf_test = rf_model.transform(test_data)
    
    # Evaluate predictions using the RMSE
    evaluator = RegressionEvaluator(labelCol="target", predictionCol="prediction", metricName="rmse")
    #rmse_train = evaluator.evaluate(rf_train)

    rmse_test = evaluator.evaluate(rf_test)

    #string = "The RMSE on the train data is {}".format(rmse_train)
    #print(string)

    string = "The RMSE on the test data is {}".format(rmse_test)
    print(string)

    # If required get feature importance
    if feature_imp:
        df_imp = get_feature_importance(rf_model.featureImportances, train_data, "features").head(10)
        display(df_imp)
        
    return rf_model

In [52]:
# If model is passed, skip model fit portion and show test results
def run_glm_model(df, cat_cols, num_cols, show_summary=True, model=False):
    stages = []
    
    # For category columns implement One Hot Encoding for each field
    for col in cat_cols:
        column_indexer = StringIndexer(inputCol=col, outputCol=f"{col}_ind")
        column_encoder = OneHotEncoderEstimator(inputCols=[f"{col}_ind"], outputCols=[f"{col}_ohe"])
        stages += [column_indexer, column_encoder]
    
    # Create a list of category fields that have been OHE
    cat_cols_ohe = [f"{col}_ohe" for col in cat_cols]
        
    # Instantiate a VectorAssembler of all categorical and number columns
    assembler = VectorAssembler(inputCols=cat_cols_ohe + num_cols, outputCol='features')
    
    # Add to stages list
    stages += [assembler]
    
    # Instantiate a pipeline with stages
    pipeline = Pipeline(stages=stages)

    # Fit the pipeline with model data frame
    pipeline_model = pipeline.fit(df)
        
    # Get test/train data
    train_data = get_train_dataset(df)
    test_data = get_test_dataset(df)

    
    # Apply the pipeline to the dataframe
    test_data = pipeline_model.transform(test_data)
    train_data = pipeline_model.transform(train_data)
        
    #test_data.show(20)
    #train_data.show(20)

    # Train a Random Forest model to predict the target field
    glm= GeneralizedLinearRegression(family= "gaussian", link="identity", maxIter=10, regParam=0.3)
    
    # if there is no model passed
    if not model:
        # Use the training data to create a model
        model = glm.fit(train_data)

    # If required get feature importance
    if show_summary:
        # Print the coefficients and intercept for generalized linear regression model
        print("Coefficients: " + str(model.coefficients))
        print("Intercept: " + str(model.intercept))

        # Summarize the model over the training set and print out some metrics
        summary = model.summary
#         print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
#         print("T Values: " + str(summary.tValues))
#         print("P Values: " + str(summary.pValues))
#         print("Dispersion: " + str(summary.dispersion))
#         print("Null Deviance: " + str(summary.nullDeviance))
#         print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
#         print("Deviance: " + str(summary.deviance))
#         print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
#         print("AIC: " + str(summary.aic))
#         print("Deviance Residuals: ")
        summary.residuals().show()
    
        
    glm_train = model.transform(train_data)
    
    # Test the model using 3 months of trip data
    glm_test = model.transform(test_data)
    
    glm_test.show(10)
    
    # Evaluate predictions using the RMSE
    evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
    #rmse_train = evaluator.evaluate(glm_train)

    rmse_test = evaluator.evaluate(glm_test)

    #string = "The RMSE on the train data is {}".format(rmse_train)
    #print(string)

    string = "The RMSE on the test data is {}".format(rmse_test)
    print(string)
        
    return model

In [43]:
# First model test- This was a prelimilary test with unclean data
category_columns = ["cat_duration", "taxi_colour", "RatecodeID"]
number_columns = ["trip_distance", "duration_mins"]

# Read data from parquet
df_model = create_model_dataframe(file_loc, category_columns, number_columns, target_field)

# Run random forest model
m1 = run_random_forest_model(df_model, category_columns, number_columns, feature_imp=True)

# test the random forest model
test_model(m1, df)

The RMSE on the test data is 42.971363475269825


,idx,name,score
0,10,trip_distance,0.423914
1,11,duration_mins,0.301928
5,3,cat_duration_ohe_Above 30 mins,0.128983
7,5,RatecodeID_ohe_1,0.054579
8,6,RatecodeID_ohe_2,0.025638
4,2,cat_duration_ohe_20-30 mins,0.018803
3,1,cat_duration_ohe_5-10 mins,0.015180
10,8,RatecodeID_ohe_3,0.015043
2,0,cat_duration_ohe_10-20 mins,0.007072
9,7,RatecodeID_ohe_5,0.005579


In [59]:
# Second model test- After major data cleaning
category_columns = ["RatecodeID"]
number_columns = ["trip_distance", "duration_mins", "passenger_count"]

# Read data from parquet
df_model = create_model_dataframe(file_loc, category_columns, number_columns, target_field)

# Run random forest model
m2 = run_random_forest_model(df_model, category_columns, number_columns, feature_imp=True)


The RMSE on the test data is 4.063293506704049


,idx,name,score
0,5,trip_distance,0.562030
1,6,duration_mins,0.204166
3,0,RatecodeID_ohe_1,0.162982
4,1,RatecodeID_ohe_2,0.053931
6,3,RatecodeID_ohe_3,0.011078
5,2,RatecodeID_ohe_5,0.005101
7,4,RatecodeID_ohe_4,0.000668
2,7,passenger_count,0.000044


In [ ]:
# Third model test to try out GLM model with EDA results
category_columns = ["RatecodeID"]
number_columns = ["trip_distance"]

# Read data from parquet
df_model = create_model_dataframe_glm(file_loc, category_columns, number_columns, target_field)

# Run random forest model
m3 = run_glm_model(df_model, category_columns, number_columns, show_summary=True)


In [49]:
run_glm_model(df_model, category_columns, number_columns, show_summary=True, model= m3)

+----------+-------------+-----+--------------+--------------+--------------------+
|RatecodeID|trip_distance|label|RatecodeID_ind|RatecodeID_ohe|            features|
+----------+-------------+-----+--------------+--------------+--------------------+
|         1|          3.7| 17.0|           0.0| (5,[0],[1.0])|(6,[0,5],[1.0,3.7...|
|         1|         1.35|12.96|           0.0| (5,[0],[1.0])|(6,[0,5],[1.0,1.3...|
|         1|          0.7| 6.95|           0.0| (5,[0],[1.0])|(6,[0,5],[1.0,0.6...|
|         1|         0.79| 7.54|           0.0| (5,[0],[1.0])|(6,[0,5],[1.0,0.7...|
|         1|          1.2| 9.35|           0.0| (5,[0],[1.0])|(6,[0,5],[1.0,1.2...|
|         1|         0.81| 7.56|           0.0| (5,[0],[1.0])|(6,[0,5],[1.0,0.8...|
|         1|         2.07| 13.5|           0.0| (5,[0],[1.0])|(6,[0,5],[1.0,2.0...|
|         1|         1.17|  9.8|           0.0| (5,[0],[1.0])|(6,[0,5],[1.0,1.1...|
|         1|         9.53| 31.8|           0.0| (5,[0],[1.0])|(6,[0,5],[1.0,

GeneralizedLinearRegression_a87d194a52d7

In [51]:
# Fourth model test to try out GLM with a different set of features
category_columns = ["RatecodeID"]
number_columns = ["duration_mins", "speed_mph"]

# Read data from parquet
df_model = create_model_dataframe_glm(file_loc, category_columns, number_columns, target_field)

# Run random forest model
m4 = run_glm_model(df_model, category_columns, number_columns, show_summary=True)

Coefficients: [-7.822053861412036,5.383078802045187,6.517288065879361,36.03416184374301,19.78894809613916,0.8132444648866406,0.661961517551023]
Intercept: 4.01573956849732
+-------------------+
|  devianceResiduals|
+-------------------+
|-1.1947683598096326|
|  1.841642121381259|
|0.23315204788496047|
| 0.5814140017314369|
|0.39756231825612254|
|  1.111530591801861|
| 0.9159718458957968|
|0.20545336534492797|
|-0.8507354406985854|
|-1.2628613879745316|
|-2.7643839958030476|
|-0.6586232308316653|
| -1.028653143952262|
| -5.735620881399413|
| -4.195183935318356|
| -1.097328917697034|
|  4.297720664225423|
| 1.5325624030928733|
|-1.4517184254238416|
| -4.558611070077987|
+-------------------+
only showing top 20 rows

The RMSE on the test data is 4.330131515586314
